In [1]:
!pip install langchain faiss-cpu pypdf GitPython openpyxl sentence-transformers transformers llama-cpp-python PyPDF2 python-dotenv streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 38.4 MB/s eta 0:00:00
     

### Notebook shows how the multiple file types can be loaded into FAISS Index (open source)

The embedding used will be again open source sentence-transformers encoders

**All Files are Converted to Text**

1) Simple Text file ,json and other pure text formats

2) tsv, csv files that are structured

3) PDF File

4) Excel File

5) Jupyter notebook Files

6) Code files

### Libraries Involved

langchain

faiss-cpu

pypdf

GitPython

openpyxl

sentence-transformers

In [2]:
from langchain.embeddings import (
    LlamaCppEmbeddings,
    HuggingFaceEmbeddings,
    SentenceTransformerEmbeddings,
    HuggingFaceInstructEmbeddings
)

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import (
    PyPDFLoader,
    DataFrameLoader,
    GitLoader
  )
import pandas as pd
import nbformat
from nbconvert import PythonExporter
import os

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub


In [4]:
def get_text_splits(text_file):
  """Function takes in the text data and returns the
  splits so for further processing can be done."""
  with open(text_file,'r') as txt:
    data = txt.read()

  textSplit = RecursiveCharacterTextSplitter(chunk_size=1000,
                                             chunk_overlap=150,
                                             length_function=len)
  doc_list = textSplit.split_text(data)
  return doc_list

In [5]:
def get_pdf_splits(pdf_file):
  """Function takes in the pdf data and returns the
  splits so for further processing can be done."""

  loader = PyPDFLoader(pdf_file)
  pages = loader.load_and_split()

  textSplit = RecursiveCharacterTextSplitter(chunk_size=1000,
                                             chunk_overlap=200,
                                             length_function=len)
  doc_list = []
  #Pages will be list of pages, so need to modify the loop
  for pg in pages:
    pg_splits = textSplit.split_text(pg.page_content)
    doc_list.extend(pg_splits)

  return doc_list

In [6]:
def embed_index(doc_list, embed_fn, index_store):
  """Function takes in existing vector_store,
  new doc_list and embedding function that is
  initialized on appropriate model. Local or online.
  New embedding is merged with the existing index. If no
  index given a new one is created"""
  #check whether the doc_list is documents, or text
  try:
    faiss_db = FAISS.from_documents(doc_list,
                              embed_fn)
  except Exception as e:
    faiss_db = FAISS.from_texts(doc_list,
                              embed_fn)

  if os.path.exists(index_store):
    local_db = FAISS.load_local(index_store,embed_fn,allow_dangerous_deserialization=True)
    #merging the new embedding with the existing index store
    local_db.merge_from(faiss_db)
    print("Merge completed")
    local_db.save_local(index_store)
    print("Updated index saved")
  else:
    faiss_db.save_local(folder_path=index_store)
    print("New store created...")


In [7]:
def get_docs_length(index_path, embed_fn):
  test_index = FAISS.load_local(index_path,
                              embeddings=embed_fn,
                              allow_dangerous_deserialization=True)
  test_dict = test_index.docstore._dict
  return len(test_dict.values())

In [8]:
# #You can change the embedding here
# embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl")

In [9]:
# pdf_docs = get_pdf_splits("/content/Full book Rich Dad Poor Dad What the Rich Teach Their Kids About Money.pdf")

# embed_index(doc_list=pdf_docs,
#             embed_fn=embeddings,
#             index_store='Rich_dad')

In [10]:
# pdf_docs = get_pdf_splits("/content/Human_nature.pdf")

# embed_index(doc_list=pdf_docs,
#             embed_fn=embeddings,
#             index_store='Human_Nature')

In [ ]:
pdf_docs = get_pdf_splits("/content/48_laws.pdf")

embed_index(doc_list=pdf_docs,
            embed_fn= HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl"),
            index_store='48_laws')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [ ]:
!wget -q -O - ipv4.icanhazip.com


In [ ]:
!streamlit run appV2.py & npx localtunnel --port 8501